In [18]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)
#====================================================================
# calling data
trans = transforms.Compose([transforms.Resize((128,128)),
                           transforms.ToTensor(),
                           transforms.Normalize((-1,-1,-1),(2,2,2))])
trainset = torchvision.datasets.ImageFolder(root="/home/gus/catvsdog",
                                           transform = trans)
train_loader = torch.utils.data.DataLoader(trainset,
                                          batch_size = 32,
                                          shuffle=True)
#====================================================================
# declaring hyperparameters
lr = 0.001
#====================================================================
# constructiong CNN model
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5,self).__init__()
        # input shape = (?, 3, 128, 128)
        # Conv -> (?, 6, 128, 128)
        # Pool -> (?, 6, 64, 64)
        self.layer1 = nn.Sequential(                   
            torch.nn.Conv2d(3,6,5,stride=1,padding=2),#C1 layer using 5x5 filters
            torch.nn.Tanh(),
            torch.nn.AvgPool2d(kernel_size=2,stride=2) #S2 layer
        )
        # input shape = (?, 6, 64, 64)
        # Conv -> (?, 16, 64, 64)
        # Pool -> (?, 16, 32, 32)
        self.layer2 = nn.Sequential(
            torch.nn.Conv2d(6,16,5,stride=1,padding=2), #C3 layer, couldn't implement exactly
            torch.nn.Tanh(),
            torch.nn.AvgPool2d(kernel_size=2,stride=2)  #S4 layer
        )
        # input shape = (?, 16, 32, 32)
        # Conv -> (?, 1, 32, 32)
        self.layer3 = nn.Sequential(
            torch.nn.Conv2d(16,1,5,stride=1,padding=2), #C5 layer
            torch.nn.Tanh()
        )
        self.layer4 = nn.Sequential(
            torch.nn.Linear(1*32*32,84,bias=True), #F6 layer
            torch.nn.Tanh(),
            torch.nn.Linear(84,2,bias=True) 
        )

        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1) # C5 layer
        out = self.layer4(out)
        return out
#====================================================================
model=LeNet5().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = lr)

total_batch = len(train_loader)

for epoch in range(10):
    avg_cost = 0
    for X,Y in train_loader:     
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/10
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 13.0982189
[Epoch:    2] cost = 13.0310555
[Epoch:    3] cost = 12.9705906
[Epoch:    4] cost = 12.9427299
[Epoch:    5] cost = 12.7701015
[Epoch:    6] cost = 12.5603266
[Epoch:    7] cost = 12.5907345
[Epoch:    8] cost = 12.433589
[Epoch:    9] cost = 12.3145685
[Epoch:   10] cost = 12.2070417


In [19]:
# calling data
trans = transforms.Compose([transforms.Resize((128,128)),
                           transforms.ToTensor(),
                           transforms.Normalize((-1,-1,-1),(2,2,2))])
testset = torchvision.datasets.ImageFolder(root="/home/gus/cattest",
                                           transform = trans)
test_loader = torch.utils.data.DataLoader(testset,
                                          batch_size = 32,
                                          shuffle=True)
with torch.no_grad():
    for X,Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        prediction = model(X)
        correct_prediction = torch.argmax(prediction,1) == Y
        accuracy = correct_prediction.float().mean()
        print('Accuracy:',accuracy.item())


Accuracy: 0.59375
Accuracy: 0.59375
Accuracy: 0.78125
Accuracy: 0.53125
Accuracy: 0.5625
Accuracy: 0.46875
Accuracy: 0.65625
Accuracy: 0.65625
Accuracy: 0.5625
Accuracy: 0.625
Accuracy: 0.65625
Accuracy: 0.4375
Accuracy: 0.65625
Accuracy: 0.5625
Accuracy: 0.71875
Accuracy: 0.71875
Accuracy: 0.65625
Accuracy: 0.625
Accuracy: 0.65625
Accuracy: 0.5
Accuracy: 0.625
Accuracy: 0.59375
Accuracy: 0.65625
Accuracy: 0.59375
Accuracy: 0.59375
Accuracy: 0.625
Accuracy: 0.59375
Accuracy: 0.5
Accuracy: 0.59375
Accuracy: 0.53125
Accuracy: 0.75
Accuracy: 0.625
Accuracy: 0.625
Accuracy: 0.59375
Accuracy: 0.59375
Accuracy: 0.5
Accuracy: 0.5625
Accuracy: 0.53125
Accuracy: 0.65625
Accuracy: 0.5
Accuracy: 0.5
Accuracy: 0.5
Accuracy: 0.6875
Accuracy: 0.59375
Accuracy: 0.46875
Accuracy: 0.59375
Accuracy: 0.34375
Accuracy: 0.625
Accuracy: 0.53125
Accuracy: 0.71875
Accuracy: 0.59375
Accuracy: 0.6875
Accuracy: 0.40625
Accuracy: 0.65625
Accuracy: 0.625
Accuracy: 0.40625
Accuracy: 0.65625
Accuracy: 0.65625
Accura

KeyboardInterrupt: 